In [ ]:
import pandas as pd
from lxml import etree
import plotly.graph_objects as go
import os

# Define the snapshot and the specifitc contingency

In [ ]:
case_name = "recollement-auto-20230222-2200-enrichi"
contg_name = "DONZAL72GOLF5"
#case_name = "recollement-auto-20230615-0500-replay"
#contg_name = "BOCTOL71N.SE1"

# Read Hades and DynaFlow files and contrsuct the voltage dataset

In [ ]:
def parse_xml_file(xml_file):
    # Parse the XML file to be able to process it later
    xml = xml_file
    parser = etree.XMLParser()
    parsed_xml = etree.parse(xml, parser)
    return parsed_xml

dwo_output_file = "./" + case_name + "/dynawo/Dynaflow_Output/" + contg_name + "/outputs/finalState/outputIIDM.xml"
parsed_xml_dynawo = parse_xml_file(dwo_output_file)
print(f"PARSED OK: {dwo_output_file}")

root = parsed_xml_dynawo.getroot()
ns = etree.QName(root).namespace
volt_dwo_dict = {"Bus_name":[], "Volt_value_dwo":[]}
for bus_i in root.iter("{%s}bus" % ns):
    volt_dwo_dict["Bus_name"].append(bus_i.attrib["id"])
    volt_dwo_dict["Volt_value_dwo"].append(float(bus_i.attrib["v"]))

df_volt = pd.DataFrame.from_dict(volt_dwo_dict)

hds_sa_file = "./" + case_name + "/hades/donneesEntreeHADES2_ANALYSE_SECU.xml"
parsed_xml_hades = parse_xml_file(hds_sa_file)
print(f"PARSED OK: {hds_sa_file}")

dict_names_hades_value = {}
root = parsed_xml_hades.getroot()
ns = etree.QName(root).namespace
dict_poste_volt_level = {}
for bus_i in root.iter("{%s}poste" % ns):
    dict_poste_volt_level[bus_i.attrib["num"]] = int(bus_i.attrib["nivTension"])

for noeud_i in root.iter("{%s}noeud" % ns):
    dict_names_hades_value[noeud_i.attrib["num"]] = [noeud_i.attrib["nom"], noeud_i.attrib["pays"], dict_poste_volt_level[noeud_i.attrib["poste"]]]

hds_output_file = "./" + case_name + "/hades/output.xml"
parsed_xml_hades = parse_xml_file(hds_output_file)
print(f"PARSED OK: {hds_output_file}")


root = parsed_xml_hades.getroot()
ns = etree.QName(root).namespace
volt_hds_dict = {"Bus_name":[], "Volt_value_hds":[], "Region":[], "Volt_level":[]}
for bus_i in root.iter("{%s}posteSurv" % ns):
    volt_hds_dict["Bus_name"].append(dict_names_hades_value[bus_i.attrib["noeud"]][0])
    volt_hds_dict["Volt_value_hds"].append(float(bus_i.attrib["vmax"]))
    volt_hds_dict["Region"].append(dict_names_hades_value[bus_i.attrib["noeud"]][1])
    volt_hds_dict["Volt_level"].append(dict_names_hades_value[bus_i.attrib["noeud"]][2])

df_volt_hds = pd.DataFrame.from_dict(volt_hds_dict)

df_volt = df_volt.set_index("Bus_name")

df_volt_hds = df_volt_hds.set_index("Bus_name")

df_volt = pd.concat([df_volt, df_volt_hds], axis=1, join="inner")

df_volt = df_volt.reset_index(drop=False)

df_volt = df_volt.sort_values(by=['Region', 'Bus_name'])

#print(df_volt)

# Calculate abs errors and pct. relative errors in bus voltages

In [ ]:
#from scipy.stats import sem 
df_volt["abs_err"] = abs(df_volt["Volt_value_hds"] - df_volt["Volt_value_dwo"])
df_volt["pct_err"] = 100*abs(df_volt["Volt_value_hds"] - df_volt["Volt_value_dwo"]) / df_volt[["Volt_value_hds", "Volt_value_dwo"]].max(axis=1)

q = 0.98
print(f"\n\n pct_err at quantile {q}:  {df_volt['pct_err'].quantile(q)}")
df_volt.sort_values("pct_err", ascending=False).head(50)  # Show the top n with the greatest error

In [ ]:
#######################################
# Function for plotting bus voltages
#######################################
def volt_plot(df):
    layout = go.Layout(autosize=False, height=600, width=1400)
    fig = go.Figure(layout=layout)
    fig.add_trace(go.Bar(x=list(df_volt_filtered.Bus_name), y=list(df_volt_filtered.Volt_value_dwo), name='DWO'))
    fig.add_trace(go.Bar(x=list(df_volt_filtered.Bus_name), y=list(df_volt_filtered.Volt_value_hds), name='HDS'))
    fig.update_layout(xaxis=dict(rangeslider=dict(visible=True)))
    fig.show()

# Voltage level 7

In [ ]:
df_volt_filtered = df_volt[df_volt.Volt_level == 7]
df_volt_filtered = df_volt_filtered[df_volt_filtered.Region == "6"]
volt_plot(df_volt_filtered)

# Voltage level 3

In [ ]:
df_volt_filtered = df_volt[df_volt.Volt_level == 3]
df_volt_filtered = df_volt_filtered[df_volt_filtered.Region == "6"]
volt_plot(df_volt_filtered)

# DynaFlow's Connections and disconnections

In [ ]:
import os
os.system(f"fgrep 'connecting' {case_name}/dynawo/Dynaflow_Output/timeLine/timeline_{contg_name}.xml")